In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/semeval_2022/maMi

/content/drive/My Drive/semeval_2022/maMi


In [3]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 18.1 MB/s 


In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

TensorFlow Version 2.7.0


In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 7
img_folder='./TRAINING/'
train_df=pd.read_csv('train.csv',sep='\t')
val_df=pd.read_csv('test.csv',sep='\t')

In [5]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
img_folder='./test/'
test_df=pd.read_csv('./test/Test.csv',sep='\t')

In [ ]:
def data_augment(image):
    p_spatial = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype = tf.float32)
    
    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    
    if p_spatial > .75:
        image = tf.image.transpose(image)
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k = 3) # rotate 270º
    elif p_rotate > .5:
        image = tf.image.rot90(image, k = 2) # rotate 180º
    elif p_rotate > .25:
        image = tf.image.rot90(image, k = 1) # rotate 90º
        
    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower = .7, upper = 1.3)
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower = .8, upper = 1.2)
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta = .1)
        
    return image

In [ ]:
#seeee class mode

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          samplewise_center = True,
                                                          samplewise_std_normalization = True,
                                                          validation_split = 0.2,
                                                          preprocessing_function = data_augment)

train_gen = datagen.flow_from_dataframe(dataframe = train_df,
                                        directory = img_folder,
                                        x_col = 'file_name',
                                        y_col = 'misogynous',
                                        subset = 'training',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = True,
                                        class_mode = 'raw',
                                        target_size = (IMAGE_SIZE, IMAGE_SIZE))

valid_gen = datagen.flow_from_dataframe(dataframe = train_df,
                                        directory = img_folder,
                                        x_col = 'file_name',
                                        y_col = 'misogynous',
                                        subset = 'validation',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = False,
                                        class_mode = 'raw',
                                        target_size = (IMAGE_SIZE, IMAGE_SIZE))

test_gen = datagen.flow_from_dataframe(dataframe = val_df,
                                        directory = img_folder,
                                        x_col = 'file_name',
                                        y_col = 'misogynous',
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = False,
                                        class_mode = 'raw',
                                        target_size = (IMAGE_SIZE, IMAGE_SIZE))


Found 6384 validated image filenames.
Found 1596 validated image filenames.
Found 2020 validated image filenames.


In [6]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                          samplewise_center = True,
                                                          samplewise_std_normalization = True)

In [7]:
test_gen = datagen.flow_from_dataframe(dataframe = test_df,
                                        directory = img_folder,
                                        x_col = 'file_name',
                                        y_col = None,
                                        batch_size = BATCH_SIZE,
                                        seed = 1,
                                        color_mode = 'rgb',
                                        shuffle = False,
                                        class_mode = None,
                                        target_size = (IMAGE_SIZE, IMAGE_SIZE))


Found 1000 validated image filenames.


In [8]:
!pip install --quiet vit-keras

from vit_keras import vit

In [ ]:
#### seee classes 1 or 2

In [ ]:
vit_model = vit.vit_b32(
        image_size = IMAGE_SIZE,
        activation = 'sigmoid',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes=1)

353263616/353253686 [==============================] - 7s 0us/step


In [ ]:
from vit_keras import visualize

x = test_gen.next()
image = x[0]

attention_map = visualize.attention_map(model = vit_model, image = image)

# Plot results
fig, (ax1, ax2) = plt.subplots(ncols = 2)
ax1.axis('off')
ax2.axis('off')
ax1.set_title('Original')
ax2.set_title('Attention Map')
_ = ax1.imshow(image)
_ = ax2.imshow(attention_map)

error: ignored

In [ ]:
model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(11, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, 'sigmoid')
    ],
    name = 'vision_transformer')

model.summary()

Model: "vision_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit-b32 (Functional)        (None, 768)               87455232  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 batch_normalization (BatchN  (None, 768)              3072      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 11)                8459      
                                                                 
 batch_normalization_1 (Batc  (None, 11)               44        
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 1)          

In [ ]:
learning_rate = 1e-4

optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, 
              loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.2), 
              metrics = ['accuracy'])

STEP_SIZE_TRAIN = train_gen.n // train_gen.batch_size
STEP_SIZE_VALID = valid_gen.n // valid_gen.batch_size

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 factor = 0.2,
                                                 patience = 2,
                                                 verbose = 1,
                                                 min_delta = 1e-4,
                                                 min_lr = 1e-6,
                                                 mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 5,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = './saved_models/vit_binary_image_checkpoint.hdf5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

callbacks = [earlystopping, reduce_lr, checkpointer]

model.fit(x = train_gen,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = valid_gen,
          validation_steps = STEP_SIZE_VALID,
          epochs = EPOCHS,
          callbacks = callbacks)
model.save('./saved_models/vit_binary_image_model.h5')

Epoch 1/7
399/399 [==============================] - ETA: 0s - loss: 0.7696 - accuracy: 0.5667 
Epoch 00001: val_accuracy improved from -inf to 0.65152, saving model to ./saved_models/vit_binary_image_checkpoint.hdf5
399/399 [==============================] - 4619s 11s/step - loss: 0.7696 - accuracy: 0.5667 - val_loss: 0.6604 - val_accuracy: 0.6515 - lr: 1.0000e-04
Epoch 2/7
399/399 [==============================] - ETA: 0s - loss: 0.6622 - accuracy: 0.6385 
Epoch 00002: val_accuracy improved from 0.65152 to 0.67866, saving model to ./saved_models/vit_binary_image_checkpoint.hdf5
399/399 [==============================] - 4493s 11s/step - loss: 0.6622 - accuracy: 0.6385 - val_loss: 0.6410 - val_accuracy: 0.6787 - lr: 1.0000e-04
Epoch 3/7
399/399 [==============================] - ETA: 0s - loss: 0.6434 - accuracy: 0.6676 
Epoch 00003: val_accuracy did not improve from 0.67866
399/399 [==============================] - 4440s 11s/step - loss: 0.6434 - accuracy: 0.6676 - val_loss: 0.6415

In [9]:
from keras.models import load_model

In [10]:
model=load_model('./saved_models/vit_binary_image_model.h5')

In [ ]:
predicted_classes = np.round(model.predict(test_gen, steps = test_gen.n // test_gen.batch_size + 1))
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())  

confusionmatrix = confusion_matrix(true_classes, predicted_classes)
plt.figure(figsize = (16, 16))
sns.heatmap(confusionmatrix, cmap = 'Blues', annot = True, cbar = True)

print(classification_report(true_classes, predicted_classes))

AttributeError: ignored

In [11]:
outputs = np.round(model.predict(test_gen, steps = test_gen.n // test_gen.batch_size + 1))
outputs=list(map(int,outputs))
predictions_db = pd.DataFrame(data=test_df['file_name'])
predictions_db['misogynist'] = outputs
predictions_db.to_csv('./final_res/answer9.txt', index=False, sep='\t', header=False)

In [ ]:
predicted_classes

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]], dtype=float32)

In [ ]:
print(classification_report(val_df.misogynous.values, predicted_classes))

              precision    recall  f1-score   support

           0       0.71      0.84      0.77       999
           1       0.81      0.67      0.73      1021

    accuracy                           0.75      2020
   macro avg       0.76      0.75      0.75      2020
weighted avg       0.76      0.75      0.75      2020



In [ ]:
from sklearn.metrics import f1_score

In [ ]:
mi=f1_score(val_df.misogynous.values,predicted_classes,average='macro')

In [ ]:
mi

0.7484806143177922